<a href="https://colab.research.google.com/github/hemangkrish7/Telecom_Churn_Analysis/blob/main/Churn_Analysis_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df =pd.read_csv('/content/drive/MyDrive/WA_Fn-UseC_-Telco-Customer-Churn.csv')


In [3]:
# --- 1. Initial Inspection ---
print("--- Initial Data Info ---")
# .info() is great for a quick overview: it shows column names, non-null counts, and data types (Dtype).
# Pay attention to any columns that might have the wrong data type.
df.info()

print("\n--- First 5 Rows ---")
# .head() shows you the first few rows to get a feel for the actual data.
print(df.head())

--- Initial Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling

In [4]:

# First, we identify rows with empty spaces and replace them. A simple approach is to
# convert them to NaN (Not a Number) and then fill with 0.
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)


In [5]:
# --- 3. Verification ---
print("\n--- Data Info After Cleaning ---")
# Let's check .info() again to confirm 'TotalCharges' is now a numeric type (like float64).
df.info()

# --- 4. Save the Cleaned Data ---
# It's a best practice to save your cleaned data to a new file.
df.to_csv('churn_cleaned.csv', index=False)

print("\n✅ Successfully cleaned the data and saved it as 'churn_cleaned.csv'")



--- Data Info After Cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  Paperles

In [6]:
import pandas as pd
import sqlite3

In [7]:
# Load the cleaned dataset
df = pd.read_csv('churn_cleaned.csv')

# Create an in-memory SQLite database
# This is a lightweight way to run SQL queries without a full database server
conn = sqlite3.connect(':memory:')

# Load the DataFrame into the SQLite database as a table named 'churn'
df.to_sql('churn', conn, index=False)

7043

In [8]:
# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# --- Now, let's write and execute our SQL queries ---

print("--- Query 1: Overall Churn Rate ---")

--- Query 1: Overall Churn Rate ---


In [9]:
# This query uses a CASE statement to create a 'Churned' column (1 for Yes, 0 for No)
# and then calculates the average to get the churn rate.
query1 = """
SELECT AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) * 100 AS ChurnRate
FROM churn;
"""
print(pd.read_sql_query(query1, conn))

   ChurnRate
0  26.536987


In [10]:
print("\n--- Query 2: Churn Rate by Contract Type ---")
# Here, we GROUP BY the contract type to see how churn differs for each.
query2 = """
SELECT
    Contract,
    AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) * 100 AS ChurnRate
FROM churn
GROUP BY Contract
ORDER BY ChurnRate DESC;
"""
print(pd.read_sql_query(query2, conn))



--- Query 2: Churn Rate by Contract Type ---
         Contract  ChurnRate
0  Month-to-month  42.709677
1        One year  11.269518
2        Two year   2.831858


In [11]:
print("\n--- Query 3: Churn Rate for Customers Without Online Security ---")
# This query filters for customers without Online Security and calculates their churn rate.
query3 = """
SELECT
    'No Online Security' AS Segment,
    AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) * 100 AS ChurnRate
FROM churn
WHERE OnlineSecurity = 'No';
"""
print(pd.read_sql_query(query3, conn))


--- Query 3: Churn Rate for Customers Without Online Security ---
              Segment  ChurnRate
0  No Online Security  41.766724


In [12]:
print("\n--- Query 4: Churn Rate by Customer Tenure ---")
# This is a powerful technique. We use a CASE statement to group customers
# into 'bins' based on how long they've been a customer. This helps
# see if new customers are more likely to churn than loyal ones.
query4 = """
SELECT
    CASE
        WHEN tenure <= 12 THEN 'New (0-12 Months)'
        WHEN tenure > 12 AND tenure <= 48 THEN 'Established (1-4 Years)'
        ELSE 'Loyal (4+ Years)'
    END AS TenureGroup,
    AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) * 100 AS ChurnRate
FROM churn
GROUP BY TenureGroup
ORDER BY ChurnRate DESC;
"""
print(pd.read_sql_query(query4, conn))


--- Query 4: Churn Rate by Customer Tenure ---
               TenureGroup  ChurnRate
0        New (0-12 Months)  47.438243
1  Established (1-4 Years)  23.644003
2         Loyal (4+ Years)   9.513176


In [13]:
print("\n--- Query 5: Churn Rate of High-Value Customers ---")
# This query filters for customers who pay more than $70/month to see
# if the company is losing its high-paying customers.
query5 = """
SELECT
    'High-Value (Charges > $70)' AS Segment,
    AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) * 100 AS ChurnRate
FROM churn
WHERE MonthlyCharges > 70;
"""
print(pd.read_sql_query(query5, conn))


--- Query 5: Churn Rate of High-Value Customers ---
                      Segment  ChurnRate
0  High-Value (Charges > $70)  35.361429


In [14]:
print("\n--- Query 6: Profile of At-Risk Customers ---")
# This query combines multiple risk factors. We are filtering for the segment
# we know is most vulnerable: Month-to-month customers who also lack Online Security.
query6 = """
SELECT
    'At-Risk: M2M & No Online Security' AS Segment,
    AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) * 100 AS ChurnRate
FROM churn
WHERE Contract = 'Month-to-month' AND OnlineSecurity = 'No';
"""
print(pd.read_sql_query(query6, conn))


--- Query 6: Profile of At-Risk Customers ---
                             Segment  ChurnRate
0  At-Risk: M2M & No Online Security   51.04523
